In [ ]:
# Cell 1: Download the model (user will fill this in)
# !wget "https://huggingface.co/haydenbanz/NeuroTumorNet/resolve/main/brain_tumor_model.h5?download=true" -O brain_tumor_model.h5

In [ ]:
# Cell 2: Import libraries and define the file upload widget
import ipywidgets as widgets
from IPython.display import display

uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False
)

display(uploader)

In [ ]:
# Cell 3: Preprocess the image and make a prediction
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
import io

def predict_tumor(uploaded_file):
    # Load the model
    model = load_model('brain_tumor_model.h5')
    
    # Get the uploaded file content
    file_content = uploader.value[0]['content']
    
    # Open the image
    img = Image.open(io.BytesIO(file_content))
    
    # Preprocess the image
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    from tensorflow.keras.applications.vgg16 import preprocess_input
    img_array = preprocess_input(img_array)
    
    # Make prediction
    prediction = model.predict(img_array)
    
    # Return the prediction
    return prediction

In [ ]:
# Cell 4: Display the prediction
def display_prediction(prediction):
    class_labels = ['Glioma Tumor', 'Meningioma Tumor', 'No Tumor', 'Pituitary Tumor']
    predicted_class_idx = np.argmax(prediction)
    confidence = np.max(prediction) * 100
    predicted_class = class_labels[predicted_class_idx]
    
    print(f"**Predicted Tumor Type:** {predicted_class}")
    print(f"**Confidence Score:** {confidence:.2f}%")

# Call the prediction and display functions when a file is uploaded
def on_upload_change(change):
    if uploader.value:
        prediction = predict_tumor(uploader.value)
        display_prediction(prediction)

uploader.observe(on_upload_change, names='value')
